In [1]:
# if run on colab
#!pip install torcheeg
#from google.colab import mount
#drive.mount('/content/drive', force_remount=True) 

In [2]:
# if run on kaggle
!pip install torcheeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
  Created wheel for torcheeg: filename=torc

In [3]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [4]:
# if run on kaggle
root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'

# if run on colab
# root_data_path = './BCICIV-2a-mat/'

In [5]:
# if run on local machine
# root_data_path = './BCICIV-2a-mat/'

In [6]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    skip_trial_with_artifacts=True,
    # offline_transform=transforms.Compose([
    #     transforms.To2d(),
    # ]),
    online_transform=transforms.Compose([
        transforms.To2d(),
        transforms.MinMaxNormalize(),
        transforms.ToTensor(),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-17 05:12:40] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./examples_pipeline/bciciv-2a.
[2024-05-17 05:12:40] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/18 [00:00<?, ?it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 1it [00:01,  1.05s/it]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 14it [00:01, 16.39it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 24it [00:01, 28.29it/s]
[RECORD /kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat/A04T.mat]: 33it [00:01, 38.61it/s]
[RECORD /kaggle/input/bci-competition

In [7]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A04T_0        A04         0       T     A04    1   
1         2254    4004    A04T_1        A04         1       T     A04    1   
2         4172    5922    A04T_2        A04         2       T     A04    1   
3         6124    7874    A04T_3        A04         3       T     A04    1   
4         8132    9882    A04T_4        A04         4       T     A04    1   
...        ...     ...       ...        ...       ...     ...     ...  ...   
4691     86751   88501  A04E_223        A04        43       E     A04    8   
4692     88657   90407  A04E_224        A04        44       E     A04    8   
4693     90585   92335  A04E_225        A04        45       E     A04    8   
4694     92699   94449  A04E_226        A04        46       E     A04    8   
4695     94758   96508  A04E_227        A04        47       E     A04    8   

      label  _record_id  
0         4   _recor

In [8]:
k_fold = KFoldGroupbyTrial(
    n_splits=4,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [9]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4
    )
    
    model = ATCNet(
        num_classes=4,
        num_electrodes=22,
        chunk_size=7*250,
    )
    
    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader, 
        max_epochs=50,
        default_root_dir=f'./examples_pipeline/atcnet_preprocessed_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-17 05:13:23] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-05-17 05:13:23] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./examples_pipeline/split for the next run, if you want to use the same setting for the experiment.
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/Convolution.cpp:1008.)
  return F.conv2d(input, weight, bias, self.stride,
2024-05-17 05:13:31.856445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 05:13:31.856546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT 

Training: |          | 0/? [00:00<?, ?it/s]

[2024-05-17 05:13:47] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.389 train_accuracy: 0.250 

[2024-05-17 05:13:54] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.379 train_accuracy: 0.289 

[2024-05-17 05:14:01] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.370 train_accuracy: 0.328 

[2024-05-17 05:14:08] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.356 train_accuracy: 0.353 

[2024-05-17 05:14:15] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.341 train_accuracy: 0.373 

[2024-05-17 05:14:22] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.326 train_accuracy: 0.396 

[2024-05-17 05:14:28] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.314 train_accuracy: 0.415 

[2024-05-17 05:14:35] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.303 train_accuracy: 0.423 

[2024-05-17 05:14:42] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.292 train_accuracy: 0.439 

[2024-05-17 05:14:49] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.282 train_accuracy: 0.457 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-17 05:19:25] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.162 test_accuracy: 0.570 



...

[2024-05-17 05:31:30] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.390 train_accuracy: 0.258 

[2024-05-17 05:31:37] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.379 train_accuracy: 0.286 

[2024-05-17 05:31:45] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.368 train_accuracy: 0.326 

[2024-05-17 05:31:52] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.357 train_accuracy: 0.354 

[2024-05-17 05:32:00] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.339 train_accuracy: 0.381 

[2024-05-17 05:32:07] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.323 train_accuracy: 0.401 

[2024-05-17 05:32:14] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.302 train_accuracy: 0.431 

[2024-05-17 05:32:22] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.286 train_accuracy: 0.453 

[2024-05-17 05:32:29] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.270 train_accuracy: 0.476 

[2024-05-17 05:32:36] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.258 train_accuracy: 0.483 



Testing: |          | 0/? [00:00<?, ?it/s]

[2024-05-17 05:37:32] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.157 test_accuracy: 0.575 



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5745097994804382     │
│         test_loss         │    1.1566580533981323     │
└───────────────────────────┴───────────────────────────┘

Fold 3 test accuracy:  0.5745


In [10]:
!mkdir weights

In [11]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))